In [1]:
import pandas as pd
import os
import json
import jieba
import re
import xlsxwriter


In [2]:
jieba.load_userdict(r'C:/Users/Big data/PycharmProjects/ourETL/user.txt')
dfall=pd.read_json(r'C:/Users/Big data/PycharmProjects/ourETL/cleard01.json')

Building prefix dict from the default dictionary ...
Loading model from cache C:\WINDOWS\TEMP\jieba.cache
Loading model cost 0.706 seconds.
Prefix dict has been built succesfully.


In [4]:
dfall=dfall.loc[sorted(dfall.index)]

In [13]:
dfall

,age,serveTime,ApplyCard,credLimit,Loan,SalPerY,holdCard,Career,ConTain
0,30,36,"[台新, Flygo, 玉山, Pi]",,0,500000,"[玉山, GoGo]",2,[心得] 台新Flygo、玉山pi 核卡時間Sat Feb 2 23:46:42 201...
1,25,2,"[中信, LINEPay, 台新, GoGo]",,0,300000,[],2,[心得] 台新黑狗悠遊卡脫白任務再度失敗時間Tue Sep 4 15:36:24 201...
2,22,,"[台新, GoGo]",20000,,260000,[],1,[心得] 學生 台新gogo核卡時間Wed Dec 26 16:10:20 2018\n\...
3,,,[小小兵信用卡],150000,2,360000,"[中信, 國泰, 台新, 玉山, 樂天信用卡, 富邦, 聯邦, 花旗, 永豐, 華南]",2,[心得] 新申辦上海2D小小兵分期卡無照會核卡時間Thu Aug 1 14:01:39 ...
4,23,48,"[花旗, 饗樂卡, 台新, GoGo, 渣打]",,3,720000,[合作金庫],2,[心得] 花旗饗樂、台新@gogo、渣打現金回饋 核卡時間Mon Feb 26 14:29...
5,,48,"[華南, 中信]",60000,0,450000,[],2,Line Pay&華南op&樂天 核卡時間Mon Feb 5 22:56:52 2018\...
6,,36,[],,0,420000,[],2,[問題] 中信LinePay較快核卡的方式？時間Sat Nov 11 18:24:04 20...
7,28,12,[中信],,1,490000,[富邦],2,[問題] 中國信託額度詢問時間Thu Jan 4 20:46:14 2018\n今天中國信...
8,,24,"[台新, GoGo]",,0,,"[富邦, 土地]",3,[問題] 台新Go Go 卡核卡失敗時間Fri Feb 9 22:34:32 2018\n...
9,24,15,[渣打],,0,360000,"[土地, 渣打, 現金回饋御璽卡]",2,[問題] 台新gogo未核卡時間Mon Oct 15 11:20:58 2018\n[職業]...


In [6]:
bank={'中信':'中信','中國信託':'中信' ,'中信金':'中信' ,'中信金控':'中信','中信銀':'中信',
'玉山':'玉山','玉山金':'玉山','玉山金控':'玉山','玉山銀':'玉山',
'Richart':'Richart', 'richart':'Richart',
'KOKO':'KOKO','koko':'KOKO',
'國泰':'國泰','國泰世華':'國泰','國泰金':'國泰','國泰金控':'國泰',
'台新':'台新','台新金':'台新','台新金控':'台新','台新銀':'台新','台新銀行':'台新',
'永豐':'永豐','永豐銀':'永豐','永豐金':'永豐','永豐金控':'永豐','永豐銀行':'永豐',
'兆豐':'兆豐','兆豐銀':'兆豐','兆豐金':'兆豐','兆豐金控':'兆豐','兆豐銀行':'兆豐',
'花旗':'花旗','花旗銀':'花旗','花旗銀行':'花旗',
'一銀':'一銀','第一銀':'一銀','第一金':'一銀','第一金控':'一銀','第一銀行':'一銀',
'富邦':'富邦','富邦金':'富邦','富邦金控':'富邦','富邦銀':'富邦','富邦銀行':'富邦',
'彰化':'彰化','彰化銀':'彰化','彰化金控':'彰化','彰化金':'彰化','彰化銀行':'彰化',
'新光':'新光','新光銀':'新光','新光金':'新光','新光金控':'新光','新光銀行':'新光',
'匯豐':'匯豐','匯豐銀行':'匯豐','匯豐銀':'匯豐',
'陽信銀行':'陽信','陽信':'陽信','陽信銀':'陽信',
'元大':'元大','元大銀行':'元大','元大金控':'元大','元大銀':'元大',
'星展':'星展','星展銀行':'星展','星展銀':'星展','DBS':'星展','dbs':'星展',
'渣打':'渣打','渣打銀行':'渣打','渣打銀':'渣打',
'上海商銀':'上海商銀','上海銀行':'上海商銀','上海商業':'上海商銀',
'華南':'華南','華南銀行':'華南','華南金控':'華南','華南銀':'華南',
'凱基':'凱基','凱基銀行':'凱基','凱基金控':'凱基',
'合作金庫':'合作金庫','合庫':'合作金庫',
'王道':'王道','王道銀行':'王道',
'澳盛':'澳盛','澳盛銀行':'澳盛','ANZ':'澳盛','anz':'澳盛',
'遠東商銀':'遠東商銀','遠銀':'遠東商銀','遠東銀':'遠東商銀',
'土地銀行':'土地','土銀':'土地',
'聯邦銀行':'聯邦','聯邦':'聯邦',
'台灣銀行':'臺灣銀行','臺灣銀行':'臺灣銀行',
'京城':'京城','京城銀行':'京城',
'日盛':'日盛','日盛銀行':'日盛',
'華泰':'華泰','華泰銀行':'華泰',
'台中銀行':'台中銀行','台中銀':'台中銀行',
'高雄銀行':'高雄銀行','高雄銀':'高雄銀行',
'台灣企銀':'臺灣企銀','臺灣企銀':'臺灣企銀','臺企銀':'臺灣企銀',
'瑞興銀行':'瑞興銀行','瑞興':'瑞興銀行',
}

behavior={
'跑步':'運動','健身房':'運動','運動':'運動',
'旅遊':'旅遊','旅行':'旅遊','機票':'旅遊','飯店':'旅遊','機場':'旅遊','出國':'旅遊',
'保險':'保險','保費':'保險',
'加油':'加油','中油':'加油',
'國外':'國外','海外':'國外',
'電影':'電影','國賓':'電影','威秀':'電影',
'網購':'網購' ,'網拍':'網購', '蝦皮':'網購', '電商':'網購','pchome':'網購','PChome':'網購','Yahoo':'網購','momo':'網購','MOMO':'網購',
'行動支付':'行動支付','PI錢包':'行動支付','pi錢包':'行動支付','街口':'行動支付','Pi錢包':'行動支付','拍錢包':'行動支付',
'新光三越':'百貨','SOGO':'百貨','sogo':'百貨','遠百':'百貨','遠東百貨':'百貨',
'紅利點數':'紅利','紅利兌換':'紅利',
'超商':'超商','7-11':'超商','全家':'超商','全家便利':'超商','萊爾富':'超商','OK便利':'超商',
'屈臣氏':'藥妝','康是美':'藥妝',
'大潤發':'量販店','家樂福':'量販店','愛買':'量販店',
'全聯':'超市','頂好':'超市',
'哩程':'航空哩程','航空哩程':'航空哩程'
}


card={
'linepay':'LINEPay','Linepay':'LINEPay','LinePay':'LINEPay','LINEPAY':'LINEPay','LINE PAY':'LINEPay','Line pay':'LINEPay',
'line pay':'LINEPay','Line':'LINEPay','LINE':'LINEPay','line':'LINEPay',
'GOGO':'GoGo','gogo':'GoGo','黑狗':'GoGo','黑狗卡':'GoGo','gogo卡':'GoGo','GOGO卡':'GoGo','Gogo卡':'GoGo','Gogo':'GoGo','GoGo':'GoGo',
'flygo':'Flygo','Flygo':'Flygo','FLYGO':'Flygo','飛狗':'Flygo','飛狗卡':'Flygo','FlyGo':'Flygo','fly':'Flygo',
'大戶':'dawho','dawho':'dawho','Dawho':'dawho','DAWHO':'dawho',
'PI':'Pi','Pi':'Pi','pi卡':'Pi','PI卡':'Pi',
'ubear':'Ubear','UBEAR':'Ubear','Ubear':'Ubear','ubear':'Ubear','bear':'Ubear','BEAR':'Ubear',
'u bear':'Ubear','U bear':'Ubear','u bear':'Ubear', 'Ubear卡':'Ubear','U Bear':'Ubear',
'only':'Only','ONLY':'Only','Only':'Only',
'賴點':'賴點卡','賴點卡':'賴點卡', 
'酷玩卡':'酷玩卡','酷玩卡':'酷玩卡',
'饗樂':'饗樂卡','饗樂卡':'饗樂卡',
'koko':'KOKO','KOKO':'KOKO','KoKo':'KOKO','Koko':'KOKO',
'星燦':'星燦卡','星燦卡':'星燦卡','星璨':'星燦卡',
'統一獅':'統一獅聯名卡',
'中信兄弟':'中信兄弟聯名卡',
'富邦悍將':'富邦悍將聯名卡',
'My樂現金回饋卡':'My樂現金回饋卡','My樂現金回饋':'My樂現金回饋卡',
'哆啦A夢JCB聯名卡':'哆啦A夢JCB聯名卡','哆啦A夢':'哆啦A夢JCB聯名卡',
'Omiyage':'Omiyage','omiyage':'Omiyage',
'尊御世界卡':'尊御世界卡',
'鑽金一卡通聯名卡':'鑽金','鑽金':'鑽金',
'炫晶御璽卡':'炫晶御璽卡','炫晶卡':'炫晶御璽卡','炫晶':'炫晶御璽卡',
'現金回饋御璽卡':'現金回饋御璽卡',
'PChome Prime聯名卡':'PChome Prime聯名卡','Prime聯名卡':'PChome Prime聯名卡','Prime':'PChome Prime聯名卡','rime':'PChome Prime聯名卡',
'購物聯名卡':'蝦皮購物聯名卡','蝦皮聯名':'蝦皮購物聯名卡','蝦皮卡':'蝦皮購物聯名卡','國泰世華蝦皮卡':'蝦皮購物聯名卡',
'Lamigo聯名卡':'Lamigo聯名卡','Lamigo':'Lamigo聯名卡','lamigo':'Lamigo聯名卡',
'SPORT卡':'SPORT卡','SPORT':'SPORT卡',
'e秒刷鈦金卡':'e秒刷鈦金卡','e秒刷':'e秒刷鈦金卡',
'小小兵信用卡':'小小兵信用卡','小小兵':'小小兵信用卡',
'everyday鈦金卡':'everyday鈦金卡','everyday':'everyday鈦金卡',
'街口聯名卡':'街口聯名卡','街口聯名':'街口聯名卡',
'Gogoro聯名卡':'Gogoro聯名卡','Gogoro':'Gogoro聯名卡','gogoro':'Gogoro聯名卡',
'現金回饋御璽卡':'現金回饋御璽卡',
'簡單卡':'簡單卡',
'數位生活':'數位生活卡',
'美國運通':'美國運通','法拉利':'法拉利無限卡',
'omiyage':'Omiyage','OMIYAGE':'Omiyage','omiyage':'Omiyage',
'e秒刷鈦金卡':'e秒刷鈦金卡','e秒刷':'e秒刷鈦金卡',
'南開科技大學認同卡':'南開科技大學認同卡',
'聯邦OTA手機信用卡':'聯邦OTA手機信用卡',
'魔法少女iPASS一卡通聯名卡':'魔法少女iPASS一卡通聯名卡',
'富邦鑽保卡':'富邦鑽保卡',
'行遍天下聯名卡':'行遍天下聯名卡',
'幸運鈦金卡':'幸運鈦金卡',
'昇恒昌聯名卡':'昇恒昌聯名卡',
'新光三越一卡通聯名卡':'新光三越一卡通聯名卡',
'悠遊晶緻卡':'悠遊晶緻卡',
'鈦金商務卡':'鈦金商務卡',
'永豐Me Card':'永豐Me Card',
'永豐鈦豐卡':'永豐鈦豐卡',
'利倍卡':'利倍卡',
'富邦財神卡':'富邦財神卡',
'八福公益卡':'八福公益卡',
'台灣銀行':'台灣銀行',
'F1加油卡':'F1加油卡',
'全家Fish悠遊聯名卡':'全家Fish悠遊聯名卡',
'新光銀行寰宇卡':'新光銀行寰宇卡',
'建築師無限卡':'建築師無限卡',
'icash聯名卡':'icash聯名卡',
'世界卡(icash)':'世界卡(icash)',
'玉山教師卡':'玉山教師卡',
'安泰銀行信用卡':'安泰銀行信用卡',
'寰宇現金回饋卡':'寰宇現金回饋卡',
'合作金庫無限卡':'合作金庫無限卡',
'合庫雙幣信用卡(日圓)':'合庫雙幣信用卡(日圓)',
'一卡通聯名卡':'一卡通聯名卡',
'TAIPEI 101 聯名卡':'TAIPEI 101 聯名卡',
'公務人員國民旅遊卡':'公務人員國民旅遊卡',
'C’est Moi我的卡':'C’est Moi我的卡',
'Super Life 卡':'Super Life 卡',
'聯邦銀行無限卡':'聯邦銀行無限卡',
'匯豐銀行白金卡':'匯豐銀行白金卡',
'樂活Visapaywave感應式白金卡':'樂活Visapaywave感應式白金卡',
'臺北科大愛校認同卡':'臺北科大愛校認同卡',
'新光三越icash聯名卡':'新光三越icash聯名卡',
'鈦金悠遊聯名卡':'鈦金悠遊聯名卡',
'公司戶商務卡':'公司戶商務卡',
'樂享晶緻卡(原大眾)':'樂享晶緻卡(原大眾)',
'華歌爾聯名卡':'華歌爾聯名卡',
'高雄going鈦金卡':'高雄going鈦金卡',
'長庚紀念醫院悠遊認同卡':'長庚紀念醫院悠遊認同卡',
'樂購卡':'樂購卡',
'雙幣信用卡':'雙幣信用卡',
'鈦金商旅卡':'鈦金商旅卡',
'大台北商銀聯名卡':'大台北商銀聯名卡',
'KOKOicash聯名卡':'KOKOicash聯名卡',
'商務卡':'商務卡',
'商旅鈦金卡商務':'商旅鈦金卡商務',
'花蓮一信聯名卡':'花蓮一信聯名卡',
'元大白金卡':'元大白金卡',
'燦坤生活一卡通聯名卡':'燦坤生活一卡通聯名卡',
'圓山大飯店一卡通聯名卡':'圓山大飯店一卡通聯名卡',
'鹿港天后宮晶緻一卡通認同卡':'鹿港天后宮晶緻一卡通認同卡',
'台塑(一卡通)聯名卡':'台塑(一卡通)聯名卡',
'ANA聯名卡':'ANA聯名卡',
'日盛銀行-無限卡':'日盛銀行-無限卡',
'台中二信聯名卡':'台中二信聯名卡',
'一銀卡御守系列':'一銀卡御守系列',
'台灣國際基督教會認同卡':'台灣國際基督教會認同卡',
'NFC 手機信用卡':'NFC 手機信用卡',
'雅芳聯名卡':'雅芳聯名卡',
'哆啦A夢晶緻卡':'哆啦A夢晶緻卡',
'NFC手機信用卡':'NFC手機信用卡',
'北港朝天宮認同卡':'北港朝天宮認同卡',
'元大鈦金卡':'元大鈦金卡',
'COSTCO聯名卡':'COSTCO聯名卡',
'元大Life卡':'元大Life卡',
'中興保全聯名卡':'中興保全聯名卡',
'合作金庫世界卡':'合作金庫世界卡',
'學學認同卡':'學學認同卡',
'哆啦A夢卡':'哆啦A夢卡',
'歐付寶悠遊聯名卡':'歐付寶悠遊聯名卡',
'一卡通聯名晶緻卡':'一卡通聯名晶緻卡',
'幸福卡':'幸福卡',
'鈦金商旅卡':'鈦金商旅卡',
'中油職工福利認同卡':'中油職工福利認同卡',
'臺灣大學卡':'臺灣大學卡',
'悠遊聯名卡':'悠遊聯名卡',
'澳盛現金回饋卡':'澳盛現金回饋卡',
'中華郵政工會認同卡':'中華郵政工會認同卡',
'彰化六信聯名卡':'彰化六信聯名卡',
'玉山雙幣信用卡':'玉山雙幣信用卡',
'京城銀行聯名卡':'京城銀行聯名卡',
'頂好超市聯名卡':'頂好超市聯名卡',
'高雄醫學院認同卡':'高雄醫學院認同卡',
'美元雙幣信用卡':'美元雙幣信用卡',
'世界卡':'世界卡',
'聯邦銀行世界卡':'聯邦銀行世界卡',
'環球無限卡':'環球無限卡',
'高雄觀光福利聯名卡':'高雄觀光福利聯名卡',
'中國信託紅利卡':'中國信託紅利卡',
'元大鑽金icash聯名卡':'元大鑽金icash聯名卡',
'佐登妮絲聯名卡':'佐登妮絲聯名卡',
'故宮之友卡':'故宮之友卡',
'媚儷鈦金卡':'媚儷鈦金卡',
'台塑(悠遊)聯名卡':'台塑(悠遊)聯名卡',
'聯邦樂活一卡通聯名卡':'聯邦樂活一卡通聯名卡',
'三信商業銀行信用卡':'三信商業銀行信用卡',
'玉山eTag悠遊聯名卡':'玉山eTag悠遊聯名卡',
'分期便利卡':'分期便利卡',
'大潤發油樂聯名卡':'大潤發油樂聯名卡',
'玫瑰卡':'玫瑰卡',
'LOVE晶緻悠遊聯名卡－寵愛紅卡':'LOVE晶緻悠遊聯名卡－寵愛紅卡',
'元大台灣霹靂卡':'元大台灣霹靂卡',
'世界商務卡':'世界商務卡',
'超級現金回饋卡':'超級現金回饋卡',
'個人商旅卡':'個人商旅卡',
'聯邦旅遊卡':'聯邦旅遊卡',
'Gogoro聯名卡':'Gogoro聯名卡',
'利high卡':'利high卡',
'淡水信用合作社聯名卡':'淡水信用合作社聯名卡',
'ｉ網購生活卡':'ｉ網購生活卡',
'哆啦A夢悠遊聯名卡':'哆啦A夢悠遊聯名卡',
'花旗現金回饋(悠遊)卡':'花旗現金回饋(悠遊)卡',
'頂好Visa payWave聯名卡':'頂好Visa payWave聯名卡',
'享購鈦金卡':'享購鈦金卡',
'台灣黑熊認同卡':'台灣黑熊認同卡',
'酷比悠遊聯名卡':'酷比悠遊聯名卡',
'玉山晶緻悠遊卡':'玉山晶緻悠遊卡',
'商旅卡':'商旅卡',
'日盛ALL PASS卡':'日盛ALL PASS卡',
'商務白金卡':'商務白金卡',
'原子小金剛悠遊聯名卡':'原子小金剛悠遊聯名卡',
'藝術白金卡':'藝術白金卡',
'iCash愛現':'iCash愛現',
'日盛GOGO加油卡':'日盛GOGO加油卡',
'雙幣鈦金商旅卡':'雙幣鈦金商旅卡',
'台南三信聯名卡':'台南三信聯名卡',
'鹿港天后宮鈦金悠遊認同卡':'鹿港天后宮鈦金悠遊認同卡',
'新光銀行分期7卡':'新光銀行分期7卡',
'Hello Kitty分享聯名卡商務':'Hello Kitty分享聯名卡商務',
'iCash聯名卡':'iCash聯名卡',
'中國人壽聯名卡':'中國人壽聯名卡',
'Combo晶片卡':'Combo晶片卡',
'國泰人壽聯名卡':'國泰人壽聯名卡',
'永豐財富無限卡':'永豐財富無限卡',
'澎湖戀戀菊島認同卡':'澎湖戀戀菊島認同卡',
'銀色之愛鈦商卡':'銀色之愛鈦商卡',
'樂活卡(原澳盛)':'樂活卡(原澳盛)',
'愛Pass鈦金卡':'愛Pass鈦金卡',
'幸福御守卡／粉':'幸福御守卡／粉',
'大甲媽祖認同卡':'大甲媽祖認同卡',
'微風廣場 JSpeedy感應式聯名卡':'微風廣場 JSpeedy感應式聯名卡',
'e秒萬事通金融信用卡':'e秒萬事通金融信用卡',
'台灣企銀商務卡':'台灣企銀商務卡',
'國民旅遊卡':'國民旅遊卡',
'金采卡':'金采卡',
'燦坤生活聯名卡':'燦坤生活聯名卡',
'運籌理財信用卡':'運籌理財信用卡',
'台北市第五信用合作社聯名卡':'台北市第五信用合作社聯名卡',
'元大樂遊卡':'元大樂遊卡',
'鑑賞家白金卡':'鑑賞家白金卡',
'大立無限卡':'大立無限卡',
'輔仁大學認同卡':'輔仁大學認同卡',
'天仁茗茶聯名卡':'天仁茗茶聯名卡',
'SunnyCard':'SunnyCard',
'華歌爾鈦金悠遊聯名卡':'華歌爾鈦金悠遊聯名卡',
'Hello Kitty點數聯名卡商務':'Hello Kitty點數聯名卡商務',
'台新財富無限卡':'台新財富無限卡',
'卡娜赫拉的小動物icash聯名卡':'卡娜赫拉的小動物icash聯名卡',
'勞動保障信用卡':'勞動保障信用卡',
'醫師尊榮無限卡':'醫師尊榮無限卡',
'里仁為美福智卡':'里仁為美福智卡',
'元大商務鈦金卡':'元大商務鈦金卡',
'新竹市民認同卡':'新竹市民認同卡',
'逢甲人一卡通鈦金認同卡':'逢甲人一卡通鈦金認同卡',
'遠傳friDay聯名卡':'遠傳friDay聯名卡',
'麗寶悠遊聯名卡':'麗寶悠遊聯名卡',
'玉山銀行卡':'玉山銀行卡',
'元大鑽金卡':'元大鑽金卡',
'無限卡':'無限卡',
'Hello Kitty鑽金聯名卡商務':'Hello Kitty鑽金聯名卡商務',
'漢神百貨聯名卡':'漢神百貨聯名卡',
'富邦數位生活卡':'富邦數位生活卡',
'國璽聯名卡':'國璽聯名卡',
'HappyCash & HAPPY GO聯名卡':'HappyCash & HAPPY GO聯名卡',
'玉山教師鈦金卡':'玉山教師鈦金卡',
'FlyGo商務':'FlyGo商務',
'白金商旅卡':'白金商旅卡',
'錢進卡':'錢進卡',
'飲食男女聯名卡':'飲食男女聯名卡',
'NISSAN感心悠遊聯名卡':'NISSAN感心悠遊聯名卡',
'中華溫馨服務卡':'中華溫馨服務卡',
'樂活悠遊鈦金卡':'樂活悠遊鈦金卡',
'京華城聯名卡':'京華城聯名卡',
'台灣企銀信用卡':'台灣企銀信用卡',
'MUJI無印良品聯名卡':'MUJI無印良品聯名卡',
'台大EMBA基金會白金卡':'台大EMBA基金會白金卡',
'富邦尊御世界卡':'富邦尊御世界卡',
'美福聯名卡':'美福聯名卡',
'玉山統一時代悠遊聯名卡':'玉山統一時代悠遊聯名卡',
'永豐國民旅遊卡':'永豐國民旅遊卡',
'新光三越聯名卡':'新光三越聯名卡',
'士林高商校友認同卡':'士林高商校友認同卡',
'NewCentury無限卡':'NewCentury無限卡',
'中醫師無限卡':'中醫師無限卡',
'旅鑽個人商務信用卡':'旅鑽個人商務信用卡',
'元大鑽金一卡通聯名卡':'元大鑽金一卡通聯名卡',
'信望愛認同卡':'信望愛認同卡',
'鼎極卡':'鼎極卡',
'華銀白金卡':'華銀白金卡',
'SPORT卡商務':'SPORT卡商務',
'鼎鑽財富無限卡':'鼎鑽財富無限卡',
'嘉義三信聯名卡':'嘉義三信聯名卡',
'愛心卡':'愛心卡',
'農金一卡通聯名卡':'農金一卡通聯名卡',
'中華航空聯名卡':'中華航空聯名卡',
'兆豐悠遊聯名卡':'兆豐悠遊聯名卡',
'雙幣鈦金商旅卡':'雙幣鈦金商旅卡',
'幸福加值卡':'幸福加值卡',
'桐花認同卡':'桐花認同卡',
'新光三越悠遊聯名卡':'新光三越悠遊聯名卡',
'清華大學悠遊認同卡':'清華大學悠遊認同卡',
'VOLVO汽車聯名卡':'VOLVO汽車聯名卡',
'亞洲無限卡':'亞洲無限卡',
'聯邦農金聯名卡':'聯邦農金聯名卡',
'亞洲微創手術中心認同卡':'亞洲微創手術中心認同卡',
'大買家油樂聯名卡':'大買家油樂聯名卡',
'台灣大哥大悠遊聯名卡':'台灣大哥大悠遊聯名卡',
'ETC聯名卡':'ETC聯名卡',
'原子小金剛卡':'原子小金剛卡',
'享利白金卡':'享利白金卡',
'Bankee卡':'Bankee卡',
'兆豐世界卡':'兆豐世界卡',
'耐斯廣場NP聯名卡':'耐斯廣場NP聯名卡',
'萬事通悠遊金融信用卡':'萬事通悠遊金融信用卡',
'富多卡':'富多卡',
'ｉ網購生活卡':'ｉ網購生活卡',
'台塑(一卡通)聯名卡商務':'台塑(一卡通)聯名卡商務',
'哆啦A夢悠遊晶緻卡':'哆啦A夢悠遊晶緻卡',
'大甲媽祖悠遊認同卡':'大甲媽祖悠遊認同卡',
'玉山雙幣信用卡':'玉山雙幣信用卡',
'獅子之友認同卡':'獅子之友認同卡',
'太陽卡':'太陽卡',
'農金悠遊聯名卡':'農金悠遊聯名卡',
'理財型白金卡':'理財型白金卡',
'東海大學悠遊認同卡':'東海大學悠遊認同卡',
'導盲犬認同卡':'導盲犬認同卡',
'桃園市市民卡聯名信用卡':'桃園市市民卡聯名信用卡',
'比漾聯名卡':'比漾聯名卡',
'Global Mall聯名卡':'Global Mall聯名卡',
'KOKO COMBO icash':'KOKO COMBO icash',
'夢時代icash 聯名卡':'夢時代icash 聯名卡',
'陽信鈦金卡':'陽信鈦金卡',
'VOGUE聯名卡':'VOGUE聯名卡',
'NFC手機信用卡':'NFC手機信用卡',
'現金回饋悠遊鈦金卡':'現金回饋悠遊鈦金卡',
'Display金融信用卡':'Display金融信用卡',
'信義房屋聯名卡':'信義房屋聯名卡',
'現金回饋悠遊鈦金Combo卡':'現金回饋悠遊鈦金Combo卡',
'微風廣場Visa Pay Wave感應式聯名卡':'微風廣場Visa Pay Wave感應式聯名卡',
'陽信櫻花白金卡':'陽信櫻花白金卡',
'微風聯名卡':'微風聯名卡',
'行動鈦金卡':'行動鈦金卡',
'玉山統一時代icash聯名卡':'玉山統一時代icash聯名卡',
'NFC寰宇鈦商手機信用卡':'NFC寰宇鈦商手機信用卡',
'家扶公益認同卡':'家扶公益認同卡',
'花旗HAPPY GO聯名卡':'花旗HAPPY GO聯名卡',
'美樂家生活卡':'美樂家生活卡',
'兆豐無限卡':'兆豐無限卡',
'手機信用卡':'手機信用卡',
'ESPRIT聯名卡':'ESPRIT聯名卡',
'樂天信用卡':'樂天信用卡','樂天':'樂天信用卡','樂天JCB':'樂天信用卡','樂天jcb':'樂天信用卡',
'晶片Combo card':'晶片Combo card',
'一卡通聯名鈦金卡':'一卡通聯名鈦金卡',
'棉花田悠遊聯名卡':'棉花田悠遊聯名卡',
'新世代信用卡':'新世代信用卡',
'中國醫藥大學暨附設醫院晶緻認同卡':'中國醫藥大學暨附設醫院晶緻認同卡',
'Combo Life卡':'Combo Life卡',
'遠東HAPPY GO頂級信用卡':'遠東HAPPY GO頂級信用卡',
'南紡夢時代聯名卡':'南紡夢時代聯名卡',
'永豐Me Display Card':'永豐Me Display Card',
'分享卡':'分享卡',
'中華電信感恩卡':'中華電信感恩卡',
'微風悠遊聯名卡':'微風悠遊聯名卡',
'經典鈦金卡':'經典鈦金卡',
'I-FIRST生活卡':'I-FIRST生活卡',
'雙幣鈦金商旅卡':'雙幣鈦金商旅卡',
'C’est Moi旅遊悠遊卡':'C’est Moi旅遊悠遊卡',
'中興大學認同卡':'中興大學認同卡',
'大葉高島屋聯名卡':'大葉高島屋聯名卡',
'TheShoppingCard分期卡':'TheShoppingCard分期卡',
'NewCentury世界卡':'NewCentury世界卡',
'新光銀行世界卡':'新光銀行世界卡',
'中華電信商務鈦金感恩卡':'中華電信商務鈦金感恩卡',
'澎湖iPASS卡':'澎湖iPASS卡',
'LOVE晶緻悠遊聯名卡－酷愛黑卡':'LOVE晶緻悠遊聯名卡－酷愛黑卡',
'合庫雙幣信用卡':'合庫雙幣信用卡',
'台塑(悠遊)聯名卡商務':'台塑(悠遊)聯名卡商務',
'富貴無限卡':'富貴無限卡',
'美安悠遊聯名卡':'美安悠遊聯名卡',
'山隆優油卡':'山隆優油卡',
'金采鈦商卡':'金采鈦商卡',
'順發多謝聯名卡':'順發多謝聯名卡',
'MAZDA天空卡':'MAZDA天空卡',
'i-Fun愛玩樂卡':'i-Fun愛玩樂卡',
'凱基無限卡':'凱基無限卡',
'板信銀行聯名卡':'板信銀行聯名卡',
'葡眾聯名卡':'葡眾聯名卡',
'幸運PLUS鈦金卡':'幸運PLUS鈦金卡',
'大紀元認同卡':'大紀元認同卡',
'富邦財神手機卡':'富邦財神手機卡',
'得利COMBO晶片卡':'得利COMBO晶片卡',
'白沙屯拱天宮媽祖認同卡':'白沙屯拱天宮媽祖認同卡',
'HappyCash & HAPPY GO聯名卡':'HappyCash & HAPPY GO聯名卡',
'民間版國民旅遊卡':'民間版國民旅遊卡',
'鈦豐卡':'鈦豐卡',
'玩全南投認同卡':'玩全南投認同卡',
'宜蘭大學悠遊認同卡':'宜蘭大學悠遊認同卡',
'聯邦國民旅遊卡':'聯邦國民旅遊卡',
'Sweet Home白金卡':'Sweet Home白金卡',
'ｉ網購生活卡':'ｉ網購生活卡',
'響樂':'響樂','饗樂':'響樂','享樂':'響樂',
'炫晶':'炫晶',
}

In [7]:
b=[]
d=[]
for n,i in enumerate(dfall['ConTain']):
    i=re.sub('已有','持有',i)
    i=re.sub('其他卡','持有',i)
    
    if '持有'in i:
        j=re.split('持有',i)[1]
        if '無' in j[0:10] or '小白'in j[0:10] or '否'in j[0:10]:
            b.append('')
            d.append(n)
            
            
        elif '['in j:
            j=re.split('\[',j)[0]
            c=j
            b.append(c)
            
        elif '?'in j: 
            k=re.split('\?',j)
            if len(str(k[0]))>6 and len(str(k[0]))<250:
                c=k[0]
                b.append(c)    
             
            elif len(str(k[1])) >10 and len(str(k[1])) <250:
                c=k[1]
                b.append(c)
            elif '過程' in j:
                j01=re.split('過程',j)[0]
                c=j01
                b.append(c)
            else:
                c=j
                b.append(c)
        elif '過程' in j:
            c=re.split('過程',j)[0]
            b.append(c)
            
        else:
            a=re.findall(r'\d/\d',j)
            if a!=[]:
                c=re.split(r'\d/\d',j)[0]
                b.append(c)
            elif '心得'in j:
                c=re.split(r'心得',j)[0]
                b.append(c)
            else:
                c=j
                b.append(c)

    else:
#         print(i)
        b.append('')
        d.append(n)


In [8]:
len(b)

3550

In [10]:

jieba.load_userdict(r'C:/Users/Big data/PycharmProjects/ourETL/user.txt')
c=[]
holdCard=[]
for wo in b:
    jb=jieba.lcut(wo)
    carlist=[]
    for w in jb:
        if w in card:
            carlist.append(card[w])
        elif w in bank:
            carlist.append(bank[w])
    
    holdCard.append(carlist)
        
print(holdCard)


[['玉山', 'GoGo'], [], [], ['中信', '國泰', '台新', '玉山', '樂天信用卡', '富邦', '聯邦', '花旗', '永豐', '華南'], ['合作金庫'], [], [], ['富邦'], ['富邦', '土地'], ['土地', '渣打', '現金回饋御璽卡'], ['富邦', '數位生活卡', '元大', '鑽金', '國泰', 'KOKO', '台新', 'GoGo'], [], ['LINEPay'], [], ['中信', '聯邦', '花旗', '永豐', '富邦', '臺灣企銀'], ['一銀', '一卡通聯名卡'], ['玉山'], [], [], [], [], ['花旗', '國泰', '無限卡', '兆豐', '個人商旅卡', '永豐', '元大鑽金卡'], ['花旗', '饗樂卡', '玉山', '永豐', 'dawho'], ['國泰', '無限卡', '國泰', 'KOKO', '國泰', '國泰', '花旗'], ['LINEPay', '中信'], ['中信', 'LINEPay'], ['渣打', '現金回饋御璽卡', '聯邦', '微風聯名卡'], ['國泰', '國泰', '元大', '鑽金'], ['國泰'], ['台新', 'GoGo', 'GoGo', '花旗', '華南', '中信', '學學認同卡', '無限卡', '世界卡', 'LINEPay', '國泰', 'KOKO', '玉山', '悠遊聯名卡', '富邦', 'LINEPay', '永豐', '美國運通', '匯豐', '現金回饋御璽卡'], ['國泰', '國泰'], [], [], [], [], [], [], ['花旗', '響樂', '玉山', '兆豐', 'e秒刷鈦金卡', '中信', 'LINEPay', '富邦', '數位生活卡'], ['花旗', '華南', '聯邦', '中信', 'LINEPay', '國泰', 'KOKO', '元大', '渣打', '現金回饋御璽卡', '星展', '台新', 'GoGo'], ['台新', 'GoGo', 'Flygo', '匯豐', '花旗', '國泰', 'KOKO', '中信', '渣打'], ['元大', '樂天信用卡'], ['元大鑽金卡', '台

In [14]:
dfall['holdCard']=holdCard

In [15]:
dfall

,age,serveTime,ApplyCard,credLimit,Loan,SalPerY,holdCard,Career,ConTain
0,30,36,"[台新, Flygo, 玉山, Pi]",,0,500000,"[玉山, GoGo]",2,[心得] 台新Flygo、玉山pi 核卡時間Sat Feb 2 23:46:42 201...
1,25,2,"[中信, LINEPay, 台新, GoGo]",,0,300000,[],2,[心得] 台新黑狗悠遊卡脫白任務再度失敗時間Tue Sep 4 15:36:24 201...
2,22,,"[台新, GoGo]",20000,,260000,[],1,[心得] 學生 台新gogo核卡時間Wed Dec 26 16:10:20 2018\n\...
3,,,[小小兵信用卡],150000,2,360000,"[中信, 國泰, 台新, 玉山, 樂天信用卡, 富邦, 聯邦, 花旗, 永豐, 華南]",2,[心得] 新申辦上海2D小小兵分期卡無照會核卡時間Thu Aug 1 14:01:39 ...
4,23,48,"[花旗, 饗樂卡, 台新, GoGo, 渣打]",,3,720000,[合作金庫],2,[心得] 花旗饗樂、台新@gogo、渣打現金回饋 核卡時間Mon Feb 26 14:29...
5,,48,"[華南, 中信]",60000,0,450000,[],2,Line Pay&華南op&樂天 核卡時間Mon Feb 5 22:56:52 2018\...
6,,36,[],,0,420000,[],2,[問題] 中信LinePay較快核卡的方式？時間Sat Nov 11 18:24:04 20...
7,28,12,[中信],,1,490000,[富邦],2,[問題] 中國信託額度詢問時間Thu Jan 4 20:46:14 2018\n今天中國信...
8,,24,"[台新, GoGo]",,0,,"[富邦, 土地]",3,[問題] 台新Go Go 卡核卡失敗時間Fri Feb 9 22:34:32 2018\n...
9,24,15,[渣打],,0,360000,"[土地, 渣打, 現金回饋御璽卡]",2,[問題] 台新gogo未核卡時間Mon Oct 15 11:20:58 2018\n[職業]...


In [16]:
dfall.to_json(r'C:/Users/Big data/PycharmProjects/ourETL/cleaC.json')
dfall.to_csv(r'C:/Users/Big data/PycharmProjects/ourETL/cleaC.csv',encoding='utf-8')
dfall.to_excel(r'C:/Users/Big data/PycharmProjects/ourETL/cleaC.xlsx')

In [223]:
d

[1,
 2,
 5,
 6,
 11,
 13,
 18,
 19,
 20,
 31,
 32,
 33,
 34,
 35,
 36,
 42,
 48,
 50,
 54,
 58,
 62,
 63,
 64,
 68,
 70,
 71,
 72,
 79,
 85,
 89,
 92,
 94,
 95,
 100,
 107,
 109,
 115,
 123,
 134,
 142,
 143,
 152,
 153,
 155,
 157,
 158,
 167,
 170,
 177,
 191,
 210,
 220,
 222,
 230,
 233,
 235,
 236,
 238,
 239,
 240,
 245,
 246,
 252,
 253,
 254,
 255,
 258,
 263,
 265,
 267,
 273,
 274,
 279,
 285,
 291,
 292,
 298,
 302,
 306,
 314,
 321,
 325,
 328,
 331,
 333,
 339,
 347,
 349,
 362,
 363,
 368,
 370,
 374,
 383,
 401,
 404,
 407,
 409,
 414,
 421,
 426,
 427,
 429,
 433,
 439,
 443,
 445,
 447,
 449,
 454,
 455,
 456,
 459,
 463,
 464,
 465,
 468,
 469,
 471,
 476,
 483,
 490,
 497,
 498,
 499,
 501,
 506,
 520,
 522,
 533,
 534,
 541,
 546,
 555,
 556,
 573,
 578,
 581,
 582,
 589,
 595,
 598,
 599,
 601,
 602,
 603,
 620,
 631,
 632,
 646,
 649,
 651,
 654,
 655,
 663,
 664,
 665,
 666,
 667,
 682,
 684,
 700,
 707,
 711,
 712,
 718,
 719,
 721,
 735,
 736,
 738,
 745,
 750,

In [222]:
c=[]
for n,a in enumerate(dfall['holdCard']):
    if a==[]:
        c.append(n)
        
print(c)

[1, 2, 5, 6, 11, 13, 17, 18, 19, 20, 31, 32, 33, 34, 35, 36, 42, 48, 50, 54, 58, 62, 63, 64, 68, 70, 71, 72, 79, 85, 86, 89, 92, 93, 94, 95, 97, 100, 107, 109, 115, 119, 123, 134, 142, 143, 148, 152, 153, 155, 157, 158, 164, 167, 170, 177, 191, 203, 210, 220, 222, 230, 233, 235, 236, 238, 239, 240, 245, 246, 252, 253, 254, 255, 258, 263, 265, 267, 273, 274, 279, 285, 291, 292, 298, 302, 306, 314, 321, 322, 325, 328, 331, 333, 339, 347, 349, 362, 363, 368, 370, 374, 383, 401, 404, 405, 407, 409, 414, 421, 426, 427, 429, 433, 439, 442, 443, 445, 447, 449, 454, 455, 456, 459, 463, 464, 465, 468, 469, 471, 476, 483, 490, 497, 498, 499, 501, 505, 506, 520, 522, 531, 533, 534, 541, 546, 555, 556, 573, 578, 581, 582, 586, 589, 595, 598, 599, 601, 602, 603, 620, 631, 632, 646, 648, 649, 651, 654, 655, 663, 664, 665, 666, 667, 682, 684, 700, 707, 711, 712, 718, 719, 721, 735, 736, 738, 745, 747, 750, 751, 759, 762, 771, 772, 776, 778, 781, 782, 784, 785, 787, 788, 800, 806, 812, 825, 834, 839, 

In [224]:
g=[]
for i in c:
    if i not in d:
        print(i)
        g.append(i)
len(g)

17
86
93
97
119
148
164
203
322
405
442
505
531
586
648
747
870
909
928
929
1017
1026
1038
1077
1083
1106
1129
1153
1156
1227
1244
1318
1414
1453
1474
1475
1502
1559
1564
1570
1573
1584
1597
1598
1606
1653
1726
1743
1744
1758
1761
1827
1908
1917
1924
1991
2006
2008
2030
2043
2056
2115
2128
2134
2144
2158
2207
2220
2339
2350
2383
2438
2458
2500
2512
2530
2594
2606
2676
2764
2766
2787
2856
2917
2944
2969
2989
2994
3040
3076
3136
3147
3148
3154
3177
3232
3272
3297
3302
3303
3304
3308
3314
3319
3334
3368
3372
3396
3416
3432
3435
3455
3512
3516


114

[心得] BoA Premium Rewards 核卡時間Tue Mar  5 12:05:53 2019
[職業]: 金融

[年齡]: 36

[年收]: NTD 120萬  (全文只有這個是台幣喔，其他的數字全是美元)

[申辦]: BoA Premium Rewards

[已持有種]: AE Platinum

[額度]: $16,000

[貸款]: No

[近三個月有無辦卡]: No

[財力證明]: No

[心得]:

 美版AE雖然很酷，可是遇到不收AE的地方還是一樣吃鱉，只能認份掏出台卡來刷。
 大家都想從銀行賺個1、2%回來，但遇到外幣消費又常常吐回去給銀行 :(
 過年回台灣後就想找個VISA、免外幣手續費的卡來辦。

 過去大家首選都是Chase Sapphire Preferred，
 可是我比較了一下，已有AE MR和台卡要兼顧，Chase的UR也不會累積太多，
 在這種情況下，不如把有需要的消費集中在MR上就好了。
 嗚嗚，第二張金屬卡byebye。


 那這張BoA Premium卡有什麼好處勒?

 1. 開卡禮: 消費$3000 現金回饋$500
 2. $100 航空退額 (到底好不好用 待大叔年中來實驗)
 3. $100 Global entry 退費
 4. 一般消費  1.5% 現金回饋
 5. 旅遊&用餐 2.0% 現金回饋


 上述現金回饋部分，還會視你在BoA往來餘額有25-75%的加碼，

 也就是說: 開卡禮最多可拿$875

           一般消費 2.625%

           旅遊用餐 3.5%


 雖有年費$95，但想像用航空退額退掉的話，就等於0年費，
 拿一張現金回饋不錯的而且又免外匯手續費的信用卡。


 申請時間表如下:

 2/14 中午網銀提出申請
      秒過

 2/20 卡片寄出

 3/04 台灣家裡收到卡片



 謝謝大家 大叔準備去刷開卡禮要的$3000囉



 剛剛急著出門覓食

 現在補上真相  https://imgur.com/a/wdUgCPg

 不是金屬卡 看起來還算有質感






In [184]:
print(g)

[17, 86, 93, 97, 119, 148, 164, 203, 322, 335, 341, 405, 442, 505, 531, 586, 629, 648, 747, 870, 909, 928, 929, 944, 995, 1017, 1026, 1038, 1077, 1083, 1103, 1106, 1129, 1153, 1156, 1227, 1244, 1318, 1414, 1453, 1474, 1475, 1502, 1559, 1564, 1570, 1573, 1584, 1597, 1598, 1606, 1653, 1726, 1743, 1744, 1758, 1761, 1827, 1908, 1917, 1924, 1991, 2006, 2008, 2030, 2043, 2056, 2115, 2128, 2134, 2144, 2158, 2207, 2220, 2339, 2350, 2383, 2438, 2458, 2500, 2512, 2530, 2594, 2606, 2676, 2764, 2766, 2787, 2856, 2917, 2944, 2969, 2989, 2994, 3040, 3076, 3136, 3147, 3148, 3154, 3177, 3232, 3272, 3297, 3302, 3303, 3304, 3308, 3314, 3319, 3334, 3368, 3372, 3396, 3416, 3432, 3435, 3455, 3512, 3516, 3524]


In [186]:
len(g)

121

In [ ]:
17

In [208]:
print(dfall['ConTain'][3432])

前情提要

[年齡] 24
[職業] 工程師
[年收] 填 1M
[貸款] 無
[財力證明] 新轉存摺
[近三個月申請] 無
[過程]
原本已有辦綠狗卡，活存 50 萬
7/25 (四) 下午接到照會電話確認填寫資料
7/26 (五) 一早發現 APP 裡的信用卡有東西了，近中午才收到簡訊跟 email 的核卡通知
[申辦] U Bear 卡 - 黑
[職業] 工程師
[年資] 6 個月
[貸款] 無
[額度] 100K
[近三個月申請] 台新 Richart 黑狗 (如上)
[已持有] 台新 Richart 黑狗 (如上)
承上
7/26 (五) 上午 Richart 黑狗核過後下午申請 U Bear

U Bear 不用等實體卡到就可以開始刷享回饋真的讚
Richart玉山UBear

